# Jupyter example for Direct Digital Synthesis (DDS) with `spcm`
_Spectrum Instrumentation GmbH (c)_

**3_jup_dds_single_static_carrier.py**

Single static carrier - This example shows the DDS functionality with 1 carrier with a fixed frequency and fixed amplitude

Example for analog replay cards (AWG) for the the M4i and M4x card-families with installed DDS option.

_See the README file in the parent folder of this examples directory for information about how to use this example._

_See the LICENSE file for the conditions under which this software may be used and distributed._

When this script is executed as a whole (e.g. using the &lt;Run All&gt; button in VS Code) the card isn't closed directly, to allow the user that make changes on-the-fly.
Only when the user executes the last cell again the card is closed. If you'd like to close the card directly, set the `close_directly` variable to True.

In [70]:
import spcm
from spcm import units # spcm uses the pint library for unit handling (units is a UnitRegistry object)

card = None
close_directly = False

## Open the card and setup the DDS mode

Uncomment the line of code that you prefer to use:
1. open a specific card using the device identifier
1. open a remote card with ip-address
1. open a card by its serial number
1. open the first card of a specific type

In [71]:
if not card:
    # card = spcm.Card('/dev/spcm0', verbose=True).open()
    # card = spcm.Card('TCPIP::192.168.1.10::inst0::INSTR', verbose=True).open()
    # card = spcm.Card(serial_number=12345, verbose=True).open()
    card = spcm.Card(card_type=spcm.SPCM_TYPE_AO, verbose=True).open()

Python version: 3.11.9 on Windows
Driver version: 7.7.23410
Kernel version: 6.3.22848
Found '/dev/spcm1': M4i.6622-x8 sn 00066


### Setup the card to do DDS generation

For the DDS generation, we set the operation mode of the card to `SPC_REP_STD_DDS`. For more information about this mode we refer to the specific manual of your device.

In [72]:
card.card_mode(spcm.SPC_REP_STD_DDS);

### Setup trigger

We'll be using the software trigger. This trigger is executed directly after the start of the card and enabling the trigger engine.

In [73]:
trigger = spcm.Trigger(card)
trigger.or_mask(spcm.SPC_TMASK_SOFTWARE);

### Setup the output channels

Set up the usage of one channel (Channel 0) in the range of +/- 500 mV. Electrical load of the channel is set to 50 Ohm.

In [74]:
channels = spcm.Channels(card, card_enable=spcm.CHANNEL0)
channels[0].enable(True)
channels[0].output_load(50 * units.ohm)
channels[0].amp(0.5 * units.V)
card.write_setup() # IMPORTANT! this turns on the card's system clock signals, that are required for DDS to work

## Setup the DDS functionality

In [75]:
dds = spcm.DDS(card, channels=channels)
dds.reset()

### Initialize the carrier
1. Set amplitude, either with power (e.g. watt or dBm) or amplitude (e.g. voltage) or relative (e.g. percentage) units.
1. Set frequency in Hz
1. Set phase in degrees, radian, etc...
1. Read back all the parameters as used by the driver
1. Print these parameters.

Finally, these parameters need to be set as soon as the card is started. Initially, the DDS firmware awaits a trigger coming from the trigger engine and executes the change of all parameters that are set before the `dds.exec_at_trg()` command. All these commands are written to the FIFO inside the DDS firmware with the `dds.write_to_card()` command.

In [76]:
# dds[0].amp(-20 * units.dBm)
dds[0].amp(100 * units.mV)
dds[0].freq(10 * units.MHz)
dds[0].phase(20 * units.degrees)

# Read back the exact parameters
freq = dds[0].get_freq(return_unit=units.MHz)
amp = dds[0].get_amp(return_unit=units.dBm)
phase = dds[0].get_phase(return_unit=units.rad)
print(f"Generated signal frequency: {freq} and amplitude: {amp} and phase: {phase}")

dds.exec_at_trg()
dds.write_to_card()

Generated signal frequency: 9.999999892897904 MHz and amplitude: -10.000530176990829 dBm and phase: 0.34907650304322624 rad


### Start the card

In [77]:
# Start command including enable of trigger engine
card.start(spcm.M2CMD_CARD_ENABLETRIGGER, spcm.M2CMD_CARD_FORCETRIGGER)

### Execute manual changes to the carrier on-the-fly
To change the carrier parameters manually, you can add commands to the FIFO queue with an `dds.exec_now()` command at the end.

In [80]:
dds[0].amp(100 * units.mV)  # Change amplitude to 100 mV
dds[0].freq(2 * units.MHz)

# Read back the exact parameters
freq = dds[0].get_freq(return_unit=units.MHz)
amp = dds[0].get_amp(return_unit=units.dBm)
phase = dds[0].get_phase(return_unit=units.rad)
print(f"Generated signal frequency: {freq} and amplitude: {amp} and phase: {phase}")

dds.exec_now()
dds.write_to_card()

Generated signal frequency: 2.0000000949949026 MHz and amplitude: -10.000530176990829 dBm and phase: 0.34907650304322624 rad


## Close the connection

IMPORTANT: at the end of usage the card needs to be manually closed.

*NOTE: when pressing &lt;Run All&gt; the stopping and closing isn't executed. Only when this cell is executed again by the user the card is stopped and closed.* 

In [79]:
if close_directly:
    card.stop()
    card.close()
    card = None
close_directly = True